In [5]:
%matplotlib inline
import numpy as np

In [6]:
def fetch(url):
  import requests, gzip, os, hashlib, numpy
  fp = os.path.join("/tmp", hashlib.md5(url.encode('utf-8')).hexdigest())
  if os.path.isfile(fp):
    with open(fp, "rb") as f:
      dat = f.read()
  else:
    with open(fp, "wb") as f:
      dat = requests.get(url).content
      f.write(dat)
  return numpy.frombuffer(gzip.decompress(dat), dtype=np.uint8).copy()
X_train = fetch("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz")[0x10:].reshape((-1, 28, 28))
Y_train = fetch("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz")[8:]
X_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz")[0x10:].reshape((-1, 28, 28))
Y_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz")[8:]


### Lets just make it in pytorch for comparision sake !

In [11]:
#Model making
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.set_printoptions(sci_mode=False)

class BobNet(torch.nn.Module):
    def __init__(self):
        super(BobNet, self).__init__()
        self.l1 = nn.Linear(784,128, bias= False)
        self.l2 = nn.Linear(128,10, bias = False)
        self.sm = nn.LogSoftmax(dim = 1)
        
    def forward(self, x):
        x = F.relu(self.l1(x))
        x = self.l2(x)
        x = self.sm(x)
        return x
    
        


### Training it in pytorch

In [13]:
model = BobNet()

"""
    you could torch.no_grad() it
    model.l1.weight.copy_(torch.tensor(layer_init(784, 128).T))
    model.l2.weight.copy_(torch.tensor(layer_init(128, 10).T))
"""


loss_function = nn.NLLLoss(reduction='none')

optim = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0)

BS = 128

losses, accuracies = [], []

for i in (t := range(1000)):
    None
